In [1]:
from glob import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import BSHTMLLoader
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os

In [2]:
folder_path = r"/Users/yeonny/okay_test/췌장암_html"
html_files = [f for f in os.listdir(folder_path) if f.endswith(".html")]
print(f"HTML 파일 개수: {len(html_files)}")

HTML 파일 개수: 42


In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 150
)

In [4]:
from charset_normalizer import detect
html_files = glob('./췌장암_html/*.html')

success_count = 0
html_docs = []

for html_file in html_files:
    try:
        # 인코딩 자동 감지
        with open(html_file, 'rb') as f:
            raw_data = f.read()
            detected_encoding = detect(raw_data)['encoding']

        # 감지된 인코딩으로 파일 로드
        loader = BSHTMLLoader(html_file, open_encoding=detected_encoding)
        documents = loader.load_and_split(text_splitter=text_splitter)
        success_count += 1
        html_docs.extend(documents)
    except Exception as e:
        print(f"Error loading HTML {html_file}: {e}")

print(f'인코딩 된 HTML파일 수: {success_count}')



인코딩 된 HTML파일 수: 42


In [5]:
# pdf 파일 ->

pdfs = glob('./췌장암_pdf/*.pdf')

pdf_docs =  []

for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    try:
        documents = loader.load()
        non_empty_docs = [doc for doc in documents if doc.page_content.strip()]
        pdf_docs.extend(non_empty_docs)
    except Exception as e:
        print(f"Error: {pdf}: {e}")

splits = text_splitter.split_documents(pdf_docs)

/opt/homebrew/anaconda3/envs/new_okay/lib/python3.9/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 0 of document ./췌장암_pdf/200207 국민암예방수칙 실천지침_췌장암.pdf
  warnings.warn(
/opt/homebrew/anaconda3/envs/new_okay/lib/python3.9/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 28 of document ./췌장암_pdf/췌장암_대한종양내과학회.pdf
  warnings.warn(


In [6]:
print(f"총 PDF 파일 수: {len(pdfs)}")
print(f"최종 로드된 non-empty 문서 수: {len(pdf_docs)}")

총 PDF 파일 수: 9
최종 로드된 non-empty 문서 수: 181


In [7]:
all_pages_content = " ".join(doc.page_content for doc in html_docs)
print(all_pages_content)

린파자 정 g] ( Lynparza tab g]) | 의약품정보 | 의료정보 | 건강정보 | 서울아산병원






































































본문바로가기







로그인
회원가입
나의차트
병원둘러보기
오시는길

Language


English
Chinese
Russian
Arabic
Mongolian
Japanese

















검색
검색어 입력
검색어를 입력하세요.



자동완성 펼치기

  

 

자동 완성 끄기











의료진/진료과
진료예약/안내
건강정보
고객서비스
의학교육
병원이야기
병원소개





로그인 하시면더욱 다양한 혜택을이용하실 수 있습니다.

로그인
나의차트
진료예약



의료진/진료과

의료진
진료과
암병원
어린이병원
심장병원
센터
클리닉
아산생명과학연구원
기타



진료예약/안내

병원안내

오시는길
주차안내
둘러보기
편의시설
전화번호안내


외래진료안내

진료예약
진료절차
진료안내


응급진료안내

진료안내
응급증상


입원진료안내

입원진료절차안내
진료/환자안전활동참여


가정간호안내

가정간호란
신청방법


검사실안내



건강정보

의료정보

인체정보
증상백과
질환백과
검사/시술/수술정보
의약품정보
알기쉬운의학용어
식사요법
환자교육자료


건강TV

내 몸 둘러보기
영상으로 보는 검사
AMC 병법
건강이야기
병원이야기
건강강좌
사람들이야기
병원뉴스


헬스에디터

메디컬칼럼
건강이야기





고객서비스

발급안내

증명서
의무기록 및 영상CD 발급
중간진료비결제
위변조문서확인


고객서비스

고객의소리
칭찬코너


이용문의

콘텐츠제공안내





의학교육

학술행사일정
학술영상다시보기



병원이야기

리얼스토리
AMC IN(人) SIGHT
사회공헌

해외의료봉사
국내의료봉사
직원봉사활동
불우환자진료비지원


후원

후원
후원현황
후원참여안내
후원이야기
나의후원내역


자원봉사 콘텐츠제공안내







In [8]:
load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
vectordb = Chroma(embedding_function=embedding, collection_name="pancreas", persist_directory="./embedding_db_path_pancreas")

In [9]:
vectordb.add_documents(html_docs)
vectordb.add_documents(splits)

['5f29d7b4-992d-45b5-90c4-db1e5752fc9f',
 'c6e2d137-7a14-4705-9751-e66ce3b4b9c7',
 'e16be20a-30d9-4e9b-9071-5bcc78f2232b',
 '07daae7e-01a2-4c23-a5f3-69de277573f3',
 '3c2b4002-834a-4892-8d18-e642d635bd61',
 '5e8bdfdd-98c7-49a1-a8a6-58e6af2c0465',
 '3d790452-b209-4777-a589-9e8bd1c2abc7',
 'ff5133a6-7f54-4e3c-862c-a91379b3fdf2',
 'dcd3a6e9-083a-46b9-8941-ecebe069b508',
 '3ea55181-785c-4a2c-8d0e-49c7b3733396',
 'c3b5fe6e-22ca-43ba-938d-9cd941301c4a',
 'ef2e9b5b-aba8-4b88-8c06-68dbbf72997b',
 'f8e663bf-0ad5-4302-bf96-f030628918a5',
 '862b92d4-b053-4447-a06f-e84b49fc8eec',
 '60c0ed6d-36ee-4d00-9216-cbe1bb5c8e73',
 'c70bf1d2-2813-446d-9951-b3e56307e60a',
 'b92baeda-e764-450e-b983-59f14a1a0d18',
 'fe63a522-03f2-4c18-b6ac-f34eb86cd0ff',
 'b3533add-ed30-4894-812e-012411f7961f',
 '12ec2f97-cf5d-4e90-8ec5-d3a754c5f57c',
 'd2e83867-be2a-4a49-9fb5-ad42418d2d92',
 '17101e6f-35c1-4466-bec7-a1ea365b7036',
 '91842796-e294-4385-9fab-5e08014dc292',
 '0f3e7d30-2fa3-47dc-88e5-cedd35007a31',
 'a83a7c6e-6fe5-

In [10]:
llm = ChatOpenAI(temperature=0, model='gpt-4o-mini')

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    llm=llm
)

In [11]:
template = """
Your response should be in JSON format.
You are an empathetic chatbot designed to provide information and support regarding diseases.
You aim to maintain a consistent and friendly style while ensuring that explanations are easy for anyone to understand.
When explaining your symptoms and how to deal with them, please answer clearly and accurately. Depending on the situation, use jargon to explain, but don't lie, be precise and detailed.
You will do best to provide appropriate answers to questions.
Your goal is to provide the answers I seek and offer the assistance.
At the end of each response, You'll provide key terms related to the question, including diseases and medications.
Put a line break after the period.
Please refrain from unnecessary words.
Please answer me only once.
don't say it over and over again
Don't use repeated phrases.
Don't rewrite the question at the end.
Don't answer anything after writing the keywords.
Don't add questions at the end.
Stop answering after listing keyword words.
Please answer in 1000 words or less.
Please must reply in Korean.
you are designed to output json.


Answer the question based only on the following context: {context}

Question: {input}
Output Format (JSON)
{{
"question": "Write the original user-submitted questions.",
"answer": "Full response to original question.",
"sources": "When writing your answer, organize it into sentences and record the context in which you wrote it.",
"source_documents": [
    {{
      "title": "The title of the source document.",
      "page": "The page number of the source document.",
      "content": "The relevant content extracted from the source document.",
      "url": "The URL or file path of the source document."
    }}]
}}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever_from_llm, "input": RunnablePassthrough()}
)

output_parser = StrOutputParser()

chain = setup_and_retrieval | prompt | llm | output_parser

In [13]:
input_query = "췌장암의 원인은 무엇인가요?"
response = chain.invoke(input_query)
print(response)

```json
{
  "question": "췌장암의 원인은 무엇인가요?",
  "answer": "췌장암의 정확한 원인은 현재까지 밝혀지지 않았습니다. 그러나 췌장암의 발생에 영향을 미치는 여러 위험 요인이 있습니다. 첫째, 유전적 요인으로는 가족성 췌장암이 있으며, 1급 친척 중 췌장암 환자가 두 명 이상일 경우 발생 위험이 6배 증가하고, 세 명일 경우 32배까지 증가합니다. 둘째, 환경적 요인으로는 흡연, 과음, 비만, 당뇨병, 만성 췌장염 등이 있습니다. 또한, 붉은 고기와 육가공품의 섭취, 유해 화학물질(농약, 유기염료 등)도 췌장암의 위험을 높이는 것으로 알려져 있습니다. 이러한 요인들은 복합적으로 작용하여 췌장암의 발생에 기여할 수 있습니다.",
  "sources": "췌장암의 원인에 대한 정보는 여러 문서에서 수집되었습니다. 유전적 요인과 환경적 요인에 대한 설명이 포함되어 있습니다.",
  "source_documents": [
    {
      "title": "췌장암 [Pancreatic cancer] | 건강정보",
      "page": "1",
      "content": "췌장암이 생기는 원인은 뚜렷하게 밝혀져 있지 않지만, 발생에 영향을 미치는 것으로 추정되는 요인들은 있습니다.",
      "url": "./췌장암_html/췌장암 [Pancreatic cancer] _ 건강정보.html"
    },
    {
      "title": "암종별 의학정보 | 서울대학교암병원",
      "page": "1",
      "content": "유전성 췌장염이나 유전성 비용종성 대장암, 가족성 선종성 용종증 등의 유전성 질환은 췌장암이 연관되어 있는 것으로 알려져 있습니다.",
      "url": "./췌장암_html/암종별 의학정보 _ 서울대학교암병원_원인.html"
    },
    {
      "title": "untitled",
  

In [14]:
print(chain.invoke('췌장암 1기와 2기 3기 차이를 알려주세요'))

```json
{
  "question": "췌장암 1기와 2기 3기 차이를 알려주세요",
  "answer": "췌장암의 병기는 암의 크기와 전이 여부에 따라 구분됩니다. \n\n1기: 암이 췌장 안에만 국한되어 있으며, 주변 장기로 전이되지 않은 상태입니다. 이 경우 수술을 통해 췌장만 절제하여 치료할 수 있습니다. \n\n2기: 암이 췌장 바깥으로 퍼져 주변 장기나 림프절로 전이되기 시작한 단계입니다. 그러나 주요 동맥 혈관에는 침범하지 않은 상태로, 여전히 수술을 통해 치료할 가능성이 있습니다. \n\n3기: 암이 주요 동맥 혈관을 침범하여 국소적으로 진행된 상태입니다. 이 경우 수술이 복잡해지며, 대부분 수술이 불가능한 상황입니다. \n\n각 병기는 암의 진행 정도에 따라 치료 방법과 예후가 달라지므로, 조기 발견과 적절한 치료가 중요합니다.",
  "sources": "췌장암의 병기 구분에 대한 정보는 암의 크기, 림프절 전이 정도, 원격 전이 여부를 기준으로 하여 설명하였습니다.",
  "source_documents": [
    {
      "title": "췌장암",
      "page": "3",
      "content": "병기 1기: 암이 췌장 안에만 국한되어 있는 경우. 병기 2기: 암이 주변 장기로 퍼져 나가기 시작하는 단계. 병기 3기: 암세포가 주요 동맥 혈관을 침범하여 수술이 불가능한 경우.",
      "url": "./췌장암_pdf/qa_data_output.pdf"
    }
  ]
}
```


In [15]:
print(chain.invoke("제 췌장암이 유전과 관련이 있을 가능성이 있나요? 가족들에게 유전자 검사를 권해야 할까요?"))

```json
{
  "question": "제 췌장암이 유전과 관련이 있을 가능성이 있나요? 가족들에게 유전자 검사를 권해야 할까요?",
  "answer": "췌장암은 유전적 요인과 관련이 있을 수 있습니다. 특히, 직계가족 중 2명 이상이 췌장암으로 진단받았다면 가족성 췌장암의 가능성이 있습니다. 그러나 현재까지 췌장암의 직접적인 원인 유전자는 발견되지 않았습니다. 유전자 검사에서 변이가 발견되지 않았더라도, 가족력이 있는 경우에는 유전자 검사를 고려하는 것이 좋습니다. 유전성 췌장염이나 포이츠-제거스 증후군과 같은 특정 유전성 증후군이 있는 경우, 조기 선별검사가 추천됩니다. 가족들에게 유전자 검사를 권하는 것은 췌장암의 위험을 평가하고 예방적 조치를 취하는 데 도움이 될 수 있습니다.",
  "sources": "이 답변은 췌장암과 관련된 유전성 암 유전자에 대한 정보를 바탕으로 작성되었습니다. 췌장암의 유전적 요인과 가족력에 대한 내용을 포함하고 있습니다.",
  "source_documents": [
    {
      "title": "untitled",
      "page": "0",
      "content": "질문: 췌장암과 관련된 유전성 암 유전자는 어떤 것이 있나요? 답변: 췌장암의 직접적인 원인유전자는 아직 발견되지 않았습니다. 췌장암의 위험이 증가되는 유전성암 유전자는 전체 췌장암 환자의 약3%에서 발견됩니다. 이러한 췌장암 관련 유전성 암유전자에는 유전성 췌장염과 연관 있는 PRSS1 변이, 점막상피 색소 침착 및 다발성의 과오종성 장 용종을 특징으로 하는 포이츠-제거스(Peutz-Jeghers) 증후군의 원인인 LKB1/STK11 변이, Familial atypical multiple mole melanoma(FAMMM) 증후군의 원인인 p16/CDKN2A 변이, 유전성 유방암과 연관이 있는 BRCA과 PALB2유전자 변이, 린치증후군의 원인인 MLH1, MSH2, MSH6 변이 등이 있습니다.",
      "url": 

In [16]:
print(chain.invoke("췌장암 치료를 위해 현재 연구 중인 새로운 약물이나 임상시험에 참여할 수 있는 방법이 있나요?"))

```json
{
  "question": "췌장암 치료를 위해 현재 연구 중인 새로운 약물이나 임상시험에 참여할 수 있는 방법이 있나요?",
  "answer": "현재 췌장암 치료를 위한 새로운 약물과 임상시험이 활발히 진행되고 있습니다. 임상시험은 새로운 치료제나 치료 방법의 효과와 안전성을 평가하기 위해 설계된 연구로, 췌장암 환자에게도 많은 기회를 제공합니다. 임상시험에 참여하기 위해서는 먼저 담당 의사와 상담하여 적합한 시험이 있는지 확인해야 합니다. 환자는 임상시험의 내용, 검사 및 치료 방법, 연구 기간, 장단점, 부작용 가능성 등을 충분히 이해한 후 참여 여부를 결정해야 합니다. 또한, 임상시험에 대한 정보는 병원이나 연구기관의 웹사이트를 통해 확인할 수 있습니다. 최근에는 췌장암 환자의 생존율을 높이기 위한 다양한 약물과 치료법이 개발되고 있으며, 이러한 연구들은 대부분 임상 연구 단계에 있습니다. 따라서, 환자들은 새로운 치료법에 대한 정보를 지속적으로 확인하고, 필요시 임상시험에 참여하는 것을 고려할 수 있습니다.",
  "sources": "췌장암 치료를 위한 임상시험과 새로운 약물 개발에 대한 정보는 주로 의학적 연구와 관련된 문서에서 확인할 수 있습니다. 임상시험은 환자의 생명 연장과 삶의 질 개선을 목표로 하며, 췌장암의 발병 원인, 예방법, 효과적인 진단 방법 등을 연구합니다.",
  "source_documents": [
    {
      "title": "췌장암 치료에 대한 연구",
      "page": "1",
      "content": "임상 시험은 사람을 대상으로 하는 의학적 연구로 임상 시험을 통해 연구자와 의사는 새로운 치료제 및 치료 방법에 대해 다음 사항들을 확인합니다. 임상 시험을 통해 의료적인 측면에서 치료 효과를 확인할 수 있으므로 중요합니다.",
      "url": "./췌장암_pdf/qa_data_output.pdf"
    },
    {
      "title": "췌장암 치료의 현재 상

In [17]:
print(chain.invoke("췌장암으로 인해 발생하는 통증이나 소화 문제를 완화하기 위해 사용할 수 있는 방법이 무엇인가요?"))

```json
{
  "question": "췌장암으로 인해 발생하는 통증이나 소화 문제를 완화하기 위해 사용할 수 있는 방법이 무엇인가요?",
  "answer": "췌장암으로 인한 통증과 소화 문제를 완화하기 위해 여러 가지 방법이 있습니다. 통증 관리에는 마약성 진통제를 사용하는 것이 일반적입니다. 이러한 진통제는 환자의 통증을 효과적으로 줄여줄 수 있습니다. 또한, 내시경 초음파나 경피적 방법으로 복강신경절에 신경 마취를 시도할 수 있습니다. 이는 통증을 유발하는 종양에 대해 방사선 치료를 병행하는 방법도 있습니다. 소화 문제에 대해서는, 췌장암 환자들은 수술 후 인슐린 분비가 줄어들어 당뇨가 발생할 수 있습니다. 이 경우, 혈당 조절을 위해 인슐린 치료가 필요합니다. 또한, 영양사와 상담하여 개인에게 적합한 식이요법을 수립하는 것이 중요합니다. 내시경적 역행성 담즙 배액술을 통해 담즙의 흐름을 개선할 수 있으며, 이는 소화 문제를 완화하는 데 도움을 줄 수 있습니다.",
  "sources": "이 답변은 췌장암 환자의 통증 및 소화 문제 관리에 대한 정보를 바탕으로 작성되었습니다.",
  "source_documents": [
    {
      "title": "췌장암 치료 및 관리",
      "page": "9",
      "content": "췌장암 환자 중 수술이 불가능하지만 전이는 없는 사람이 40% 정도 되는데, 이들에게는 방사선치료를 시행할 수 있습니다. 방사선치료와 함께 항암제를 투여하면 생존 기간이 연장되는 경우가 많습니다. 또한 수술 중에 방사선을 조사하기도 합니다. 췌장암은 아주 심한 통증을 유발하기 때문에 마약성 진통제를 사용해야 할 때가 많습니다. 이럴 경우엔 내시경 초음파나 경피적으로 주사바늘을 넣어 복강신경절의 신경 마취를 시도할 수 있습니다.",
      "url": "./췌장암_pdf/160818_췌장암.pdf"
    },
    {
      "title": "췌장암 진단 및 치료",
      "

In [18]:
print(chain.invoke("췌장암이 당뇨병과 관련이 있다는 이야기를 들었는데, 제 혈당 관리가 암 치료에 어떤 영향을 미칠 수 있나요?"))

```json
{
  "question": "췌장암이 당뇨병과 관련이 있다는 이야기를 들었는데, 제 혈당 관리가 암 치료에 어떤 영향을 미칠 수 있나요?",
  "answer": "췌장암과 당뇨병은 밀접한 관계가 있습니다. 췌장은 인슐린을 생성하는 기관으로, 췌장암이 진행되면 췌장의 기능이 저하되어 당뇨병이 발생할 수 있습니다. 연구에 따르면, 췌장암 환자의 약 2/3에서 3/4가 혈당이 높아져 있는 것으로 나타났습니다. 이는 췌장암이 혈당 조절에 영향을 미칠 수 있음을 시사합니다. 또한, 췌장암 수술 후 일부 환자에서 혈당이 호전되는 경우도 관찰되고 있습니다. 따라서 혈당 관리는 췌장암 치료에 중요한 요소가 될 수 있습니다. 혈당이 잘 조절되면 전반적인 건강 상태가 개선되고, 치료의 효과를 높일 수 있습니다. 그러나 모든 환자에게 동일하게 나타나는 것은 아니므로, 개인의 상태에 따라 다를 수 있습니다. 혈당 조절을 위해서는 식이요법과 인슐린 요법이 필요할 수 있으며, 의사와 상담하여 적절한 관리 방법을 찾는 것이 중요합니다.",
  "sources": "췌장암과 당뇨병의 관계에 대한 정보는 췌장암 환자의 증상 관리와 관련된 내용을 바탕으로 작성되었습니다.",
  "source_documents": [
    {
      "title": "췌장암 수술 후 당뇨병의 관리 | 건강TV | 건강정보 | 서울아산병원",
      "page": "1",
      "content": "당뇨병은 췌장암의 위험요인으로 알려져 있으며, 췌장암이 진행되면 당뇨병이 발생할 수 있습니다. 췌장암 환자의 2/3~3/4가 혈당이 높아져 있으며, 수술 후 혈당이 호전되는 경우도 있습니다.",
      "url": "./췌장암_html/췌장암 수술 ᒪ팡의 관리 _ 건강TV _ 건강정ᅩ _ 서울아산병원.html"
    }
  ]
}
```


In [19]:
print(chain.invoke("치료 과정 중에도 일상생활을 최대한 유지하고 생활의 질을 높이기 위해 추천할 만한 활동이나 프로그램이 있을까요?"))

```json
{
  "question": "치료 과정 중에도 일상생활을 최대한 유지하고 생활의 질을 높이기 위해 추천할 만한 활동이나 프로그램이 있을까요?",
  "answer": "치료 과정 중에도 일상생활을 최대한 유지하고 생활의 질을 높이기 위해 여러 가지 활동과 프로그램을 추천합니다. 우선, 식생활과 관련된 여러 증상을 잘 관리하는 것이 중요합니다. 피곤하거나 지치는 느낌이 들 때는 자신에게 관대해지고, 필요할 때 충분히 쉬는 것이 필요합니다. 주변의 도움을 고맙게 받아들이고, 도움이 필요하다고 이야기하는 것을 두려워하지 마세요. 일상에서 집중할 수 있는 작은 일들을 계획하는 것도 좋습니다. 예를 들어, 다음 주에 즐거운 일을 계획하거나, 간단한 일상적인 활동을 통해 가능한 정상적인 생활을 유지하는 것이 도움이 됩니다. 스트레스, 통증, 불안을 줄이기 위해 따뜻한 목욕, 깊게 숨쉬기, 편안한 음악 듣기와 같은 간단한 방법들을 활용할 수 있습니다. 가벼운 운동도 체력과 건강을 지키는 데 도움이 되며, 운동량과 시간은 기분 상태에 따라 조절해야 합니다. 운동 계획을 세우기 전에는 의사나 간호사와 상담하는 것이 좋습니다. 또한, 마사지, 명상, 이완요법과 같은 보완요법도 통증 및 부작용을 줄이고 불안을 다스리는 데 도움이 됩니다.",
  "sources": "이 답변은 췌장암 환자의 치료 과정 중 일상생활을 유지하고 생활의 질을 높이기 위한 방법에 대한 내용을 바탕으로 작성되었습니다.",
  "source_documents": [
    {
      "title": "췌장암 환자의 식이요법과 대처 방법",
      "page": "20",
      "content": "식생활과 관련된 여러 증상을 잘 관리하는 것이 육체적, 정서적으로 건강 상태를 조절하는 데 중요합니다. 여러 증상과 부작용을 관리할 수 있으면 기분이 더 좋아질 수 있고 건강 상태를 조절하는 데 도움이 될 것입니다.",
      "url": "./췌장암_pdf/췌장암_대하